# Part 8: Train an Encrypted NN on Encrypted Data

In this notebook, we're going to use all the techniques we've learned thus far to perform neural network training (and prediction) while both the model and the data are encrypted.

Note that this is Autograd is not *yet* supported for encrypted variables, thus we'll have to roll our own gradients ourselves.

# Step 1: Create Workers and Toy Data

In [1]:
import syft as sy
hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)

In [2]:
# create our dataset
data = sy.FloatTensor([[0,0],[0,1],[1,0],[1,1]])
target = sy.FloatTensor([[0],[0],[1],[1]])

In [3]:
model = sy.zeros(2,1)

In [4]:
data


 0  0
 0  1
 1  0
 1  1
[syft.core.frameworks.torch.tensor.FloatTensor of size 4x2]

# Step 2: Encrypt the Model and Data

Encryption here comes in two steps. Since Secure Multi-Party Computation only works on Longs, in order to operate over numbers with decimal points (such as weights and activations), we need to encode all of our numbers using Fixed Precision, which will give us several bits of floating point precision. We do this by calling .fix_precision().

We can then call .share() as we have for other demos, which will encrypt all of the values by sharing them between Alice and Bob.

In [5]:
data = data.fix_precision().share(alice, bob)
target = target.fix_precision().share(alice, bob)
model = model.fix_precision().share(alice, bob)

# Step 3: Train

And now we can train using simple tensor logic. Note that autograd is not yet supported (but it will be in the Torch 1.0 refactor which you can [watch here](https://github.com/OpenMined/PySyft/issues/1587))

In [24]:
for i in range(10):
    pred = data.mm(model)
    grad = pred - target
    update = data.transpose(0,1).mm(grad)

    model = model - update * 0.1
    loss = grad.get().decode().abs().sum()
    print(loss)
    


2.0
1.6000000014901161
1.2999999821186066
1.0720000118017197
0.896000012755394
0.7599999904632568
0.6499999910593033
0.5619999840855598
0.4880000092089176
0.42799999564886093
